# 12.05 - Ética y Buenas Prácticas en Scraping

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 20 min

---

## ¿Qué aprenderás?

- Leer e interpretar `robots.txt`
- Respetar rate limits y no saturar servidores
- Aspectos legales básicos del scraping
- Alternativas al scraping cuando existen
- Checklist antes de scrapear un sitio

---

## 1. robots.txt: las reglas del sitio

Todo sitio web puede incluir un archivo `robots.txt` en su raíz que indica qué rutas pueden visitar los bots. Respetarlo es una obligación ética y en muchos casos legal.

In [ ]:
import requests
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse


def can_fetch(url: str, user_agent: str = '*') -> bool:
    """
    Comprueba si robots.txt permite scrapear una URL.

    Parameters
    ----------
    url : str
        URL que queremos scrapear
    user_agent : str
        Nuestro user agent

    Returns
    -------
    bool
        True si está permitido, False si no
    """
    parsed = urlparse(url)
    robots_url = f"{parsed.scheme}://{parsed.netloc}/robots.txt"

    rp = RobotFileParser()
    rp.set_url(robots_url)

    try:
        rp.read()
        return rp.can_fetch(user_agent, url)
    except Exception:
        # Si no hay robots.txt, asumimos que está permitido
        return True


# Ejemplos
test_urls = [
    "https://es.wikipedia.org/wiki/Madrid",
    "https://api.citybik.es/v2/networks",
]

for url in test_urls:
    allowed = can_fetch(url)
    status = '✓ Permitido' if allowed else '✗ Bloqueado'
    print(f"{status}: {url}")

In [ ]:
# Ver el contenido del robots.txt de Wikipedia
response = requests.get(
    "https://es.wikipedia.org/robots.txt",
    headers={'User-Agent': 'DataPortfolio/1.0'}
)

# Mostrar las primeras líneas
for line in response.text.split('\n')[:30]:
    print(line)

---

## 2. Rate limiting: no saturar el servidor

In [ ]:
import time
import random


class RateLimiter:
    """
    Limita la velocidad de peticiones a un servidor.
    Añade variación aleatoria para parecer más humano.
    """

    def __init__(self, min_delay: float = 1.0, max_delay: float = 3.0):
        """
        Parameters
        ----------
        min_delay : float
            Espera mínima en segundos entre peticiones
        max_delay : float
            Espera máxima en segundos entre peticiones
        """
        self.min_delay = min_delay
        self.max_delay = max_delay
        self._last_request = 0.0

    def wait(self) -> None:
        """Espera el tiempo necesario antes de la siguiente petición."""
        elapsed = time.time() - self._last_request
        delay = random.uniform(self.min_delay, self.max_delay)

        if elapsed < delay:
            time.sleep(delay - elapsed)

        self._last_request = time.time()


def scrape_urls(urls: list[str]) -> list[str]:
    """Scraping de múltiples URLs respetando rate limits."""
    limiter = RateLimiter(min_delay=1.0, max_delay=2.0)
    headers = {'User-Agent': 'DataPortfolio/1.0 (educational)'}
    results = []

    for i, url in enumerate(urls, 1):
        limiter.wait()  # Esperar antes de cada petición
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            results.append(response.text)
            print(f"[{i}/{len(urls)}] OK: {url}")
        except requests.exceptions.RequestException as e:
            print(f"[{i}/{len(urls)}] Error: {e}")
            results.append(None)

    return results


# Ejemplo
urls = [
    "https://api.citybik.es/v2/networks/bicimad",
    "https://api.citybik.es/v2/networks/valenbisi",
]

responses = scrape_urls(urls)
print(f"\nRespuestas recibidas: {sum(1 for r in responses if r is not None)}/{len(urls)}")

---

## 3. Aspectos legales

> **Aviso:** esto es orientación general, no asesoría legal. Consulta a un abogado para casos específicos.

| Situación | Consideración |
|---|---|
| Datos públicos sin login | Generalmente permitido |
| Datos detrás de login | Puede violar los ToS |
| Datos personales (GDPR) | Requiere base legal |
| ToS dice "no scraping" | Alto riesgo legal |
| Uso comercial de los datos | Verifica licencia |
| Reproducir datos con copyright | Prohibido |

### Alternativas al scraping

Antes de scrapear, comprueba si existe:
1. **API oficial** del sitio
2. **Dataset descargable** (CSV, JSON, XML)
3. **Portal de Open Data** (datos.gob.es, datos.madrid.es)
4. **RSS feed** para noticias y blogs

---

## 4. Checklist antes de scrapear

In [ ]:
def scraping_checklist(url: str) -> dict[str, bool]:
    """
    Ejecuta comprobaciones básicas antes de scrapear un sitio.

    Parameters
    ----------
    url : str
        URL objetivo

    Returns
    -------
    dict[str, bool]
        Resultado de cada comprobación
    """
    parsed = urlparse(url)
    base = f"{parsed.scheme}://{parsed.netloc}"
    headers = {'User-Agent': 'DataPortfolio/1.0 (educational)'}
    results = {}

    # 1. ¿Permite robots.txt?
    results['robots_txt_permite'] = can_fetch(url)

    # 2. ¿Tiene API pública?
    api_indicators = ['/api/', '/v1/', '/v2/', 'api.']
    results['posible_api_disponible'] = any(ind in url for ind in api_indicators)

    # 3. ¿El sitio responde?
    try:
        resp = requests.get(url, headers=headers, timeout=5)
        results['sitio_accesible'] = resp.status_code == 200
    except Exception:
        results['sitio_accesible'] = False

    # 4. ¿Tiene sitemap?
    try:
        sitemap = requests.get(f"{base}/sitemap.xml", headers=headers, timeout=5)
        results['tiene_sitemap'] = sitemap.status_code == 200
    except Exception:
        results['tiene_sitemap'] = False

    return results


url_test = "https://api.citybik.es/v2/networks/bicimad"
checks = scraping_checklist(url_test)

print(f"Checklist para: {url_test}\n")
for check, result in checks.items():
    icon = '✓' if result else '✗'
    print(f"  {icon} {check.replace('_', ' ').capitalize()}")

---

## Resumen: reglas de oro del scraping ético

| Regla | Por qué |
|---|---|
| Lee siempre `robots.txt` | Es la política del sitio |
| Usa un `User-Agent` descriptivo | Transparencia con el servidor |
| Añade delays entre peticiones | No saturar el servidor |
| Usa variación aleatoria en delays | No parecer un bot agresivo |
| Revisa los ToS | Evitar problemas legales |
| Prefiere APIs a HTML | Más estable y legal |
| No scrapes datos personales | GDPR y privacidad |
| Cachea los datos | No repetir peticiones innecesarias |

---

**Anterior:** [12.04 - Selenium](./12_04_selenium.ipynb)  
**Inicio del módulo:** [12.01 - Requests Basics](./12_01_requests_basics.ipynb)